In [9]:
import pandas as pd
import numpy as np
# from random import shuffle
from DiyLSTMmod import *
from sklearn.model_selection import train_test_split
from optimizationAlgorithm import *
import shutil
from sklearn.utils import shuffle
tf.random.set_seed(123)
shutil.rmtree('models')  
os.mkdir('models')
print(tf.__version__)
print(keras.__version__)
import random 
from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

dataset_path = './dataset/Pistachio_28_Features_Dataset.xlsx'
test_path = './dataset/test.xlsx'

2.2.0
2.3.1


In [10]:
def FReIndex(colIndex,reIndex):
    dirindex = {}
    lisindex = []
    for i in range(len(colIndex)):
        lisindex.append(i)
    dirindex = dict(zip(lisindex,colIndex))
    colIndex = []
    for i in range(len(reIndex)):
        for key in dirindex:
            if int(key) + 1 == int(reIndex[i]):
                colIndex.append(dirindex[key])
                break
    #print(colIndex)
    return colIndex
def creat_dataset_X_Y(lis,path,test_size):
    df = pd.read_excel(path)
    df = shuffle(df)
    df.reset_index(drop = True,inplace = True)
    #print(df)
    Lisclass = df['Class'].values
    LisClassNp = []
    for i in range(len(Lisclass)):
        if (Lisclass[i] == 'Kirmizi_Pistachio'):
            LisClassNp.append([0,1])
        else:
            LisClassNp.append([1,0])
    Y = np.array(LisClassNp)
    df = df.drop(columns='Class')
    #colIndex = ['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity','ConvexArea', 'Extent', 'Perimeter']
    colIndex = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
       'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
       'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
       'Shapefactor_3', 'Shapefactor_4']#开心果
    colIndex = FReIndex(colIndex,lis)
    df = df[colIndex]
    X = df.values
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    #拆分训练集和测试集
    x_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=40)
    X_train = X
    Y_train = Y
    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

    X_train = tf.cast(X_train, dtype='float32')
    X_test = tf.cast(X_test, dtype='float32')
    Y_train = tf.cast(Y_train, dtype='float32')
    Y_test = tf.cast(Y_test, dtype='float32')
    return X_train,Y_train,X_test,Y_test
def enSimpleCode(lis):
    #print('lis',lis)
    X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,dataset_path,test_size=0.0001)
    checkpoint_path = "./models_Raisin/{categorical_accuracy:.5f}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    train_acc = []
    valid_acc = []
    for i in range(1):
        t,v = k_fold(10, X_train, Y_train,100,128)
        train_acc.append(t)
        valid_acc.append(v)
    #shutil.rmtree('./models')
    Note = open('restultOptimization.txt',mode='a+')
    Note.write(f'{lis}' + ',' + f'{valid_acc}\n')
    Note.close()
    # print('训练集：',train_acc)
    # print('验证集:',valid_acc)

    # print('训练集平均，验证集平均：',ave_list(train_acc),ave_list(valid_acc))
    return valid_acc
def get_pred_target_lis(pre,tar):
    pred_list = []
    target_list = []
    for i in pre:  #[0,1]:0   [1,0]:1
        if(i[0]>0.5):
            pred_list.append(1)
        elif(i[1]>0.5):
            pred_list.append(0)
        else:
            print("0.5 and 0.5")
            pred_list.append(0)
    #print(len(tar))
    for j in range(len(tar)):
        #print(i)
        if(tar[j][0]==0):
            target_list.append(0)
        elif(tar[j][0]==1):
            target_list.append(1)
        else:
            print('Y_train error!')
    return pred_list,target_list

In [11]:
lis = [x for x in range(1,17)]
print(lis)
#lis = [8, 10, 6, 2, 11, 7, 12, 1, 5, 4, 13, 14, 15, 9, 16, 3]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [12]:
#enSimpleCode(lis)
#开心果  11.25  [0.8782383322715759]

In [13]:
# def enSimpleCodeLis(lis):
#     net2 =  my_model((sequence_lenth,input_size), name='MyModelMLP2')
#     net2.compile(loss='categorical_crossentropy',#categorical_crossentropy', binary_crossentropy
#         optimizer='adam', metrics=['categorical_accuracy'],run_eagerly=True)  
#     net2.load_weights('./0.88229.ckpt')
#     X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,test_path,test_size=0.999)
#     history = net2.evaluate(X_test,Y_test,verbose=0)
#     Note = open('restultOptimization.txt',mode='a+')
#     Note.write(f'{lis}' + ',' + f'{history[1]}\n')
#     Note.close()
#     return int(-history[1]*10000)

In [14]:
# Note = open('restultOptimization.txt',mode='w')
# Note.close()

In [15]:
class Tabu:
    def __init__(self,tabulen=100,preparelen=200,parlen=16):
        #self.costroad = costroad
        self.tabulen=tabulen
        self.parlen = parlen
        self.preparelen=preparelen
        #self.city,self.cityids,self.stid=self.loadcity2()  #我直接把他的数据放到代码里了
        self.route=self.randomroute()
        self.tabu=[]
        self.prepare=[]
        self.curroute=self.route.copy()
        self.bestcost=self.costroad(self.route)
        self.bestroute=self.route
    def costroad(self,lis):
        #print('lis :',lis)
        net2 =  my_model((sequence_lenth,input_size), name='MyModelMLP2')
        net2.compile(loss='categorical_crossentropy',#categorical_crossentropy', binary_crossentropy
            optimizer='adam', metrics=['categorical_accuracy'],run_eagerly=True)  
        net2.load_weights('./0.89458.ckpt')
        X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,test_path,test_size=0.00001)
        #print('xtest,',X_train)
        f = net2(X_train)
        #print(Y_train)
        pred_list,target_list = get_pred_target_lis(f,Y_train)
        f1 = f1_score(y_true=target_list, y_pred=pred_list, average='binary') # 也可以指定micro模式
        acc = accuracy_score(y_true=target_list, y_pred=pred_list)
        rs = recall_score(y_true=target_list,y_pred=pred_list,average='binary') # 也可以指定micro模式
        op = confusion_matrix(y_true=target_list, y_pred=pred_list)
        print('op: ',op)
        #print(f)
        history = net2.evaluate(X_train,Y_train,verbose=0)
        # Note = open('restultOptimization.txt',mode='a+')
        # Note.write(f'{lis}' + ',' + f'{history[1]}\n')
        # Note.close()
        #print('history :',history)
        print('f1: ',f1,'acc: ',acc,'rs: ',rs,'restult: ',history[1])
        return int(-history[1]*10000)
    def randomroute(self):
        parlen = self.parlen
        rt = [x for x in range(1,parlen+1)]
        random.shuffle(rt)
        return rt
    def randomswap2(self,route):
        #随机交换路径的两个节点
        route=route.copy()
        while True:
            a=random.choice(route)
            b=random.choice(route)
            if a==b or a==1 or b==1:
                continue
            ia=route.index(a)
            ib=route.index(b)
            route[ia]=b
            route[ib]=a
            return route
    def step(self):
        #搜索一步路找出当前下应该搜寻的下一条路
        rt=self.curroute

        i=0
        #print('----------------1---------------')
        while i<self.preparelen:     #产生候选路径
            prt=self.randomswap2(rt)
            if int(self.costroad(prt)) not in self.tabu:    #产生不在禁忌表中的路径
                self.prepare.append(prt.copy())
                i+=1
        c=[]
        #print('----------------2---------------')
        for r in self.prepare:
            c.append(self.costroad(r))
        mc=min(c)
        mrt=self.prepare[c.index(mc)]     #选出候选路径里最好的一条
        if mc<self.bestcost:
            self.bestcost=mc
            self.bestroute=mrt.copy()      #如果他比最好的还要好，那么记录下来
        self.tabu.append(mc)#int(mrt))    #这里本来要加 mrt的 ，可是mrt是路径，要对比起来麻烦，这里假设每条路是由长度决定的
                                    #也就是说 每个路径和他的长度是一一对应，这样比对起来速度快点，当然这样可能出问题，更好的有待研究
        self.curroute=mrt   #用候选里最好的做下次搜索的起点
        self.prepare=[]
        if len(self.tabu)>self.tabulen:
            self.tabu.pop(0)

In [16]:
import timeit
# lis = [x for x in range(1,8)]
# print(lis)
#t=Tabu(enSimpleCodeLis(lis))
t = Tabu()
print('ok')
# print(t.city)
print(t.route)
print(t.bestcost)
print(t.curroute)
for i in range(20):
    t.step()
    #if i%50==0:
    print(t.bestcost)
    print(t.bestroute)
    print(t.curroute)
print('ok')
#print(timeit.timeit(stmt="t.step()", number=1000,globals=globals()))
print('ok')

op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75
ok
[9, 2, 16, 7, 11, 12, 3, 10, 6, 13, 8, 15, 14, 5, 1, 4]
-7500
[9, 2, 16, 7, 11, 12, 3, 10, 6, 13, 8, 15, 14, 5, 1, 4]
op:  [[ 8  2]
 [ 0 10]]
f1:  0.9090909090909091 acc:  0.9 rs:  1.0 restult:  0.8999999761581421
op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75
op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75
op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75
op:  [[6 4]
 [2 8]]
f1:  0.7272727272727272 acc:  0.7 rs:  0.8 restult:  0.699999988079071
op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75
op:  [[2 8]
 [1 9]]
f1:  0.6666666666666667 acc:  0.55 rs:  0.9 restult:  0.550000011920929
op:  [[6 4]
 [1 9]]
f1:  0.7826086956521738 acc:  0.75 rs:  0.9 restult:  0.75


KeyboardInterrupt: 

-10000
[11, 5, 2, 3, 1, 13, 14, 8, 7, 12, 9, 6, 10, 15, 4, 16]
[8, 10, 6, 2, 11, 7, 12, 1, 5, 4, 13, 14, 15, 9, 16, 3]